In [1]:
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

## Estimating Parameter Using SMM

In [2]:
data=np.loadtxt('/Users/rubyzhang/Desktop/UChicago/OSML/BootCamp2017/Econ/Wk4_StrEst/data/MacroSeries.txt',delimiter=',')
ct = data[:,0]
kt = data[:,1]
wt = data[:,2]
rt = data[:,3]

In [3]:
def vcorrcoef(X,Y):
    '''Returns a vector of the column by column covariance of two matrices'''
    Xm = np.reshape(np.mean(X,axis=0),(1,X.shape[1]))
    Ym = np.reshape(np.mean(Y,axis=0),(1,Y.shape[1]))
    r_num = np.sum((X-Xm)*(Y-Ym),axis=0)
    r_den = np.sqrt(np.sum((X-Xm)**2,axis=0)*np.sum((Y-Ym)**2,axis=0))
    r = r_num/r_den
    return r

def simulate_moments(init_kt,alpha,beta,rho,mu,sigma,T,S):
    # Draw TxS matrix of all normally distributed errors
    et = np.random.normal(loc=0,scale=(sigma**2),size=(T,S))
    kt = np.empty_like(et)
    zt = np.empty_like(et)
    for i in range(T):
        if i==0:
            kt[i,:] = init_kt
            zt[i,:] = mu
        else:
            zt[i,:] = rho*zt[i-1,:]+(1-rho)*mu+et[i,:]
            kt[i,:] = alpha*beta*np.exp(zt[i-1,:])*(kt[i-1,:]**alpha)
    wt = (1-alpha)*np.exp(zt)*(kt**alpha)
    rt = alpha*np.exp(zt)*(kt**(alpha-1))
    ct = wt+rt*kt
    ct[:-1,:] = ct[:-1,:]-kt[1:,:]
    
    # Scalar values
    mean_ct = np.mean(ct)
    mean_kt = np.mean(kt)
    
    # Vector of length S
    var_ct = np.var(ct,axis=0)
    var_kt = np.var(kt,axis=0)
    
    # Vector of length S 
    corr_ct_kt = vcorrcoef(ct,kt)
    
    # Vector of length S-1
    corr_kt = vcorrcoef(kt[:-1,:],kt[1:,:])
    
    data_moms = np.array([mean_ct,mean_kt,np.mean(var_ct),
                          np.mean(var_kt),np.mean(corr_ct_kt),np.mean(corr_kt)])
        
    return data_moms

def err_vec(wt,kt,ct,rt,alpha,beta,rho,mu,sigma,T,S,simple=False):
    mean_ct = np.mean(ct)
    mean_kt = np.mean(kt)
    var_ct = np.var(ct)
    var_kt = np.var(kt)
    corr_ct_kt = vcorrcoef(np.array([ct]).T,np.array([kt]).T)[0]
    corr_kt = vcorrcoef(np.array([kt[:-1]]).T,np.array([kt[1:]]).T)[0]
    
    moms_model = simulate_moments(mean_kt,alpha,beta,rho,mu,sigma,T,S)
    moms_data = np.array([mean_ct,mean_kt,var_ct,var_kt,corr_ct_kt,corr_kt])
    
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    return err_vec

def criterion(params, *args):
    alpha,beta,rho,mu,sigma = params
    wt,kt,ct,rt,T,S = args
    W = np.eye(6)
    err = err_vec(wt,kt,ct,rt,alpha,beta,rho,mu,sigma,T,S)
    crit_val = np.dot(np.dot(err.T, W), err) 
    return crit_val

In [5]:
alpha_init = 0.5
beta_init = 0.9
rho_init = 0.8
mu_init = -0.2
sigma_init = 0.022
T = 100
S = 1000
params_init = np.array([alpha_init, beta_init, rho_init, mu_init, sigma_init])
args = (wt,kt,ct,rt,T,S)
bounds = ((0.01,0.99),(0.01,0.99),(-0.99,0.99),(-0.5,1),(0.001,1))
results = opt.minimize(criterion, params_init, args=args, bounds=bounds)
alpha_MLE, beta_MLE, rho_MLE, mu_MLE, sigma_MLE= results.x
print(results)
print('alpha_MLE=', alpha_MLE, 'beta_MLE=', beta_MLE, 'rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE, 'sigma_MLE', sigma_MLE)
print('Minimized Criterion:', criterion(results.x, *args))

      fun: 3.6628823184253023
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([   5.82554063, -100.6711225 ,  -60.54640576, -126.0900651 ,
       -115.97018177])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 144
      nit: 2
   status: 0
  success: True
        x: array([ 0.7951161 ,  0.98999999,  0.98999997,  0.27915655,  0.11959156])
alpha_MLE= 0.795116104868 beta_MLE= 0.989999985078 rho_MLE= 0.989999968274 mu_MLE= 0.279156546415 sigma_MLE 0.119591563142
Minimized Criterion: 3.66288165791
